C++ presentation of 'Templet application' interface and model
=============================================================

```
-----------------------------------------
TEMPLET_APP commands |  base CLI commands 
---------------------|-------------------
write_event          |  event        
read_events          |  log          
reply_on_query       |  reply        
write_query          |  query        
read_answer          |  answer
-----------------------------------------       

TOKEN(NAME,INFO)
EVENT(ORDINAL,TAG,EXTERN,NAME,DATA)
ANSWER(ORDINAL,NAME,DATA)

write_event(TOKEN,TAG,DATA)   --> ORDINAL \/ false
read_events(TOKEN,ORDINAL)    --> EVENT[] \/ false
write_query(TOKEN,TAG,DATA)   --> ORDINAL \/ false
reply_on_query(TOKEN,ORDINAL,DATA) --> true \/ false
read_answer(TOKEN,ORDINAL)    --> DATA \/ false
```

In [2]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // service interface    

typedef string   NAME;
typedef string   PERMISSION; // permission info
typedef string   DATA;       // abstract binary data
typedef unsigned ORDINAL;
typedef unsigned TAG;
typedef bool     EXTERN;

#define ASSERT(expr)  if(!(expr))return false;

bool has_write_event(PERMISSION);
bool has_read_events(PERMISSION);
bool has_reply_on_query(PERMISSION);
bool has_client_actions(PERMISSION);

struct TOKEN{
    NAME name;             // who is the token owner?
    PERMISSION permission; // what permissions does it have?
}; 

struct ANSWER;

struct EVENT{
    EVENT(ORDINAL _ord,TAG _tag,EXTERN _ext, NAME _name,DATA _data){
        ord=_ord; tag=_tag; ext=_ext; name=_name; data=_data; answer=0;}
    ORDINAL ord;
    TAG     tag;
    EXTERN  ext;  // ext==true used for client queries
    NAME    name;
    DATA    data; // if ext==true, data may include query GUID
    ANSWER* answer;
};

struct ANSWER{
    ANSWER(NAME _name,DATA _data){name=_name; data=_data;}
    NAME    name; // who wrote the answer?
    DATA    data;
};

/*-1-*/
bool write_event(TOKEN tkn,TAG tag,DATA data,ORDINAL& ord){  
    ASSERT(has_write_event(tkn.permission));
    EVENT ev(new_event_num,tag,/*extern*/false,tkn.name,data);      
    ord = new_event_num++;
    events.push_back(ev);
    return true;
}
    
/*-2-*/              
bool read_events(TOKEN tkn,ORDINAL ord,list<EVENT>& evs){
    ASSERT(has_read_events(tkn.permission));
    evs.clear(); 
    for(EVENT ev:events) if(ev.ord>=ord) evs.push_back(ev);
    return true;
}

/*-3-*/
bool write_query(TOKEN tkn,TAG tag,DATA data,ORDINAL& ord){
    ASSERT(has_client_actions(tkn.permission));
    EVENT ev(new_event_num,tag,/*extern*/true,tkn.name,data);
    ord = new_event_num++;
    events.push_back(ev);
    return true;
}

/*-4-*/
bool reply_on_query(TOKEN tkn,ORDINAL ord,DATA data){
    ASSERT(has_reply_on_query(tkn.permission));
    for(EVENT& ev:events)
    if(ev.ord==ord){
        if(ev.answer)return true;
        ANSWER* ans = new ANSWER(tkn.name,data);      
        ev.answer = ans;
        return true;
    }   
    return false;
}    

/*-5-*/            
bool read_answer(TOKEN tkn,ORDINAL ord,DATA& data){
    ASSERT(has_client_actions(tkn.permission));
    for(EVENT ev:events)
    if( ev.ord==ord && ev.name==tkn.name && ev.ext && ev.answer){ 
        data = ev.answer->data;
        return true;
    }
    return false;
}

private: // service state
    
list<EVENT>  events;
ORDINAL new_event_num = 0;

};

```
Интерфейс утилиты, реализующей агент доступа к сервису синхронизации состояния распределенного приложения
=========================================================================================================

Идея: обеспечить возможность реализации сервиса синхронизации глобального состояния распределенного приложения на основе "готовой" системы с функцией удаленного запуска задач (Everest, HTCondor, Dirac, PanDA и т.д.), а также собственной системы, специально оптимизированной для выполения псевдозадач синхронизации состояния с использованием описываемой утилиты; выполнить компактную эталоную реализацию сервиса синхронизации состояния на одном узле в разделяемой памяти, полностью описывающую семантику команд (задач) сервиса и используемую для реализации сервиса на базе "готовых" систем с функцией удаленного запуска задач.

Форматы
=======
<file>     формат определяется в приложении
<base>     адрес для подключения к сервису или SQLite файл базы данных, хранящей состояние приложения в виде
           последовательности событий
<logfile>  запись фрагмента последовательности событий обновления состояния, внутренний формат
<info>     информация о пользователе в виде, требуемом конкретной реализацией сервиса
<num>      целое число в десятичном представлении

Параметры
=========

Обязательные
------------
(--base | -b) <base>     база данных состояния приложения
(--user | -u) <info>     информация о пользователе приложения, обращающегося к логу событий

Дополнительные
--------------
(--tag  | -t) <num>      тэг события
(--ord  | -о) <num>      порядковый номер события
(--extra| -x)            при чтении лога событий также записывать в <logfile> ответы на события-запросы

Команды
=======

Серверные
---------
(--event | -e) <file>    запись события file с тэгом tag, в stdout - порядковый номер события
(--log   | -l) <logfile> запись выборки из лога событий в файл logfile, начиная с заданного ord
(--reply | -r) <file>    запись ответа file на запрос с указанным номером ord

Клиентские
----------
(--query | -q) <file>    подать запрос с данными file и тэгом tag, в stdout - порядковый
                         номер события-запроса
(--answer| -a) <file>    считать ответ на запрос с порядковым номером ord в файл file

Реализации
----------
tlite          реализация на базе встроенной СУБД SQLite - эталонная реализация и "псевдозадача" (см.выше)
tsync          реализация на базе специализированного оптимизированного сервиса на базе СУБД PostgreSQL
tmini + tserv  распределенная реализация на базе SQLite, tserv - серверная часть, tmini - клиентская часть
tever + tlite  реализация на базе платформы Everest - реализация для сравнения производительности с tsync
tcond + tlite  реализация на базе платформы HTCondor - для исследования применимости подхода в типовом грид-кружении
tdira + tlite  реализация на базе платформы DIRAC - популярная грид-система в области ядерной физики
tpand + tlite  реализация на базе платформы PanDA - популярная грид-система в области ядерной физики
```

```
Почему не требуется хранить файлы (большие блоки данных) как часть событий/запросов/ответов
===========================================================================================

У агентов c доступом на добавление файлов можно создать буферы для начальной загрузки файлов. Эти буферы должны быть ограниченного размера. Только что загруженный файл помещается в буфер и уменьшает размер доступного для загрузки файлового пространства у агента. Если другой специальный агент выполнит операцию перемещения файла в хранилище приложения, файл освобождает буфер агента. Если агент не может загрузить файл из-за переполнения буфера, он либо ждет опустошения буфера, либо сам освобождает буфер специальной командой.

То есть агенты со специальными полномочиями подтверждают, что информация о загруженном файле попала в приложение, файл будет использоваться. В противном случае потребовалось бы помещать информацию о загруженных в буфер файлах в событие явно (как при запуске задач в системе Эверест) и перемещать файлы из временного буфера в хранилище приложения при записи события (или ответа).

Описанные меры позволяют контролировать объем хранилища приложения: в хралилище попадают только те файлы, информация о которых содержится в логе приложения. Буферы же пользователей - временные области небольшого фиксированного размера, которые можно безопасно удалить.
``` 